In [40]:
import numpy as np
from sympy import *
from sympy.matrices import Matrix

First create the symbols

In [41]:
# Create symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') #theta_i
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

Then create modified DH parameters

In [42]:
### KUKA KR210 ###
# Create Modified DH parameters
s = {alpha0:     0,  a0:      0, d1:  0.75,  
     alpha1: -pi/2,  a1:   0.35, d2:     0, q2: q2-pi/2, 
     alpha2:     0,  a2:   1.25, d3:     0,       
     alpha3: -pi/2,  a3: -0.054, d4:  1.50,
     alpha4:  pi/2,  a4:      0, d5:     0,
     alpha5: -pi/2,  a5:      0, d6:     0,
     alpha6:     0,  a6:      0, d7: 0.303, q7: 0}


Define the transformation matrix

In [43]:
# Define Modified DH Transformation matrix
# base_link to link1
T0_1 = Matrix([[             cos(q1),            -sin(q1),            0,              a0],
               [ sin(q1)*cos(alpha0), cos(q1)*cos(alpha0), -sin(alpha0), -sin(alpha0)*d1],
               [ sin(q1)*sin(alpha0), cos(q1)*sin(alpha0),  cos(alpha0),  cos(alpha0)*d1],
               [                   0,                   0,            0,               1]])
T0_1 = T0_1.subs(s)

T1_2 = Matrix([[             cos(q2),            -sin(q2),            0,              a1],
               [ sin(q2)*cos(alpha1), cos(q2)*cos(alpha1), -sin(alpha1), -sin(alpha1)*d2],
               [ sin(q2)*sin(alpha1), cos(q2)*sin(alpha1),  cos(alpha1),  cos(alpha1)*d2],
               [                   0,                   0,            0,               1]])
T1_2 = T1_2.subs(s)

T2_3 = Matrix([[             cos(q3),            -sin(q3),            0,              a2],
               [ sin(q3)*cos(alpha2), cos(q3)*cos(alpha2), -sin(alpha2), -sin(alpha2)*d3],
               [ sin(q3)*sin(alpha2), cos(q3)*sin(alpha2),  cos(alpha2),  cos(alpha2)*d3],
               [                   0,                   0,            0,               1]])
T2_3 = T2_3.subs(s)

T3_4 = Matrix([[             cos(q4),            -sin(q4),            0,              a3],
               [ sin(q4)*cos(alpha3), cos(q4)*cos(alpha3), -sin(alpha3), -sin(alpha3)*d4],
               [ sin(q4)*sin(alpha3), cos(q4)*sin(alpha3),  cos(alpha3),  cos(alpha3)*d4],
               [                   0,                   0,            0,               1]])
T3_4 = T3_4.subs(s)
T4_5 = Matrix([[             cos(q5),            -sin(q5),            0,              a4],
               [ sin(q5)*cos(alpha4), cos(q5)*cos(alpha4), -sin(alpha4), -sin(alpha4)*d5],
               [ sin(q5)*sin(alpha4), cos(q5)*sin(alpha4),  cos(alpha4),  cos(alpha4)*d5],                                                                                                           
               [                   0,                   0,            0,               1]])
T4_5 = T4_5.subs(s)
T5_6 = Matrix([[             cos(q6),            -sin(q6),            0,              a5],
               [ sin(q6)*cos(alpha5), cos(q6)*cos(alpha5), -sin(alpha5), -sin(alpha5)*d6],
               [ sin(q6)*sin(alpha5), cos(q6)*sin(alpha5),  cos(alpha5),  cos(alpha5)*d6],                                                                                                           
               [                   0,                   0,            0,               1]])
T5_6 = T5_6.subs(s)
T6_G = Matrix([[             cos(q7),            -sin(q7),            0,              a6],                                                                                                           
               [ sin(q7)*cos(alpha6), cos(q7)*cos(alpha6), -sin(alpha6), -sin(alpha6)*d7],
               [ sin(q7)*sin(alpha6), cos(q7)*sin(alpha6),  cos(alpha6),  cos(alpha6)*d7],
               [                   0,                   0,            0,               1]])
T6_G = T6_G.subs(s)

Correct the Orientation difference between the definition of the gripper_link in the urdf-file and the Denavit-Hardenberg Convention.

First apply a body-fixed rotation about the z-axis followed by a body-fixed rotation about the y-axis.

R_corr is the composition of those two rotations.

In [44]:
#Correct the Gripper
R_z =  Matrix([[     cos(np.pi), -sin(np.pi),             0, 0],
               [     sin(np.pi),  cos(np.pi),             0, 0],
               [              0,           0,             1, 0],
               [              0,           0,             0, 1]])

R_y =  Matrix([[  cos(-np.pi/2),           0, sin(-np.pi/2), 0],
               [              0,           1,             0, 0],
               [ -sin(-np.pi/2),           0, cos(-np.pi/2), 0],
               [              0,           0,             0, 1]])

R_corr = simplify(R_z * R_y)

Create individual transformation matrices

In [45]:
# Create individual transformation matrices
T0_2 = simplify(T0_1 * T1_2) # base_link to link_2
T0_3 = simplify(T0_2 * T2_3) # base_link to link_3
T0_4 = simplify(T0_3 * T3_4) # base_link to link_4
T0_5 = simplify(T0_4 * T4_5) # base_link to link_5
T0_6 = simplify(T0_5 * T5_6) # base_link to link_6
T0_G = simplify(T0_6 * T6_G) # base_link to gripper_link
T_total = simplify(T0_G * R_corr) # Total HT between base_link and gripper_link with orientation correction applied

#T0_G = simplify(T0_1 * T1_2 * T2_3 * T3_4 * T4_5 * T5_6 * T6_G)
sub_dict = { q1: -2.77, q2: 0.36, q3: -2.41, q4: 4.56, q5: 1.31, q6: 4.70 }
sub_dict_zero = { q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0 }
# Evaluate transforms
print ("T0_1 = ", T0_1.evalf(subs=sub_dict_zero),"\n")
print ("T0_2 = ", T0_2.evalf(subs=sub_dict_zero),"\n")
print ("T0_3 = ", T0_3.evalf(subs=sub_dict_zero),"\n")
print ("T0_4 = ", T0_4.evalf(subs=sub_dict_zero),"\n")
print ("T0_5 = ", T0_5.evalf(subs=sub_dict_zero),"\n")
print ("T0_6 = ", T0_6.evalf(subs=sub_dict_zero),"\n")
print ("T0_G = ", T0_G.evalf(subs=sub_dict_zero),"\n")
print  ("T_total = ", T_total.evalf(subs=sub_dict_zero),"\n")

T0_1 =  Matrix([[1.00000000000000, 0, 0, 0], [0, 1.00000000000000, 0, 0], [0, 0, 1.00000000000000, 0.750000000000000], [0, 0, 0, 1.00000000000000]]) 

T0_2 =  Matrix([
[  0, 1.0,   0, 0.35],
[  0,   0, 1.0,    0],
[1.0,   0,   0, 0.75],
[  0,   0,   0,  1.0]]) 

T0_3 =  Matrix([
[  0, 1.0,   0, 0.35],
[  0,   0, 1.0,    0],
[1.0,   0,   0,  2.0],
[  0,   0,   0,  1.0]]) 

T0_4 =  Matrix([
[  0,    0, 1.0,  1.85],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]]) 

T0_5 =  Matrix([
[  0, 1.0,   0,  1.85],
[  0,   0, 1.0,     0],
[1.0,   0,   0, 1.946],
[  0,   0,   0,   1.0]]) 

T0_6 =  Matrix([
[  0,    0, 1.0,  1.85],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]]) 

T0_G =  Matrix([
[  0,    0, 1.0, 2.153],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]]) 

T_total =  Matrix([
[                  1.0,                     0, 6.12323399573677e-17, 2.153],
[-7.49879891330929e-33,                   1.0, 1.224

Extract End-Effector position and orient from request
Calculate joint angles using Geometric IK method

In [10]:
# Extract end-effector position and orientation from request
# Test Input
gt_1,gt_2,gt_3 = symbols('gt_1:4') # gt_1 = Gripper x, gt_2 = Gripper y, gt_3 = Gripper z
gr_1, gr_2, gr_3 = symbols('gr_1:4') # Rotation Values in degrees - roll, pitch, yaw

find the location of the WC relative to the base frame

In [46]:
# Build a rotation matrix from the Roll, Pitch and Yaw angles
def get_wrist_rot_matrix(orientation):
    roll  = orientation[0]
    pitch = orientation[1]
    yaw   = orientation[2]
    
    Rrpy = Matrix([[    cos(yaw)*cos(pitch),   cos(yaw)*sin(pitch)*sin(roll) - sin(yaw)*cos(roll),    cos(yaw)*sin(pitch)*cos(roll) + sin(yaw)*sin(roll)],
                   [    sin(yaw)*cos(pitch),   sin(yaw)*sin(pitch)*sin(roll) + cos(yaw)*cos(roll),    sin(yaw)*sin(pitch)*cos(roll) - cos(yaw)*sin(roll)],
                   [            -sin(pitch),             cos(pitch)*sin(roll),                                       cos(pitch)*cos(roll)               ]])

    print(Rrpy)
    return Rrpy


def get_wrist_pos(position, Rrpy):
       
    px = position[0]
    py = position[1]
    pz = position[2]
    
    end_effector_length = 0.453
    
    
    lx = Rrpy[ 0, 0 ]
    ly = Rrpy[ 1, 0 ]
    lz = Rrpy[ 2, 0 ]

    print("lx:",lx," ly:",ly," lz:",lz)

    d6 = 0

    # Calculate Wrist Center
    wx = px - ( end_effector_length + d6 ) * lx
    wy = py - ( end_effector_length + d6 ) * ly
    wz = pz - ( end_effector_length + d6 ) * lz

    print( 1.9466)
    print("wx:",wx," wy:",wy," wz:",wz)
    
    return wx, wy, wz, Rrpy


In [47]:
Rrpy = get_wrist_rot_matrix([0, 0, 0])
wx, wy, wz, Rrpy = get_wrist_pos([2.3029, 0, 1.9466], Rrpy)

Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
lx: 1  ly: 0  lz: 0
1.9466
wx: 1.84990000000000  wy: 0  wz: 1.94660000000000


find joint variables, q1, q2 and q3, such that the WC has coordinates equal to equation (3). This is the hard step. One way to attack the problem is by repeatedly projecting links onto planes and using trigonometry to solve for joint angles. Unfortunately, there is no generic recipe that works for all manipulators so you will have to experiment.

In [51]:
# Calculate joint angles using Geometric IK method
def get_t123 (wx, wy, wz):
    
    theta1 = atan2(wy, wx)
    a_1 = 0.35
    a_2 = 1.25
    d_1 = 0.75
    d_4 = 1.50
    a_3 = -0.054
    
    
    wc_dist_x = sqrt(wx**2 + wy**2) - a_1 # Subtract a1 as horizontal offset from robot-base
    wc_dist_z = wz - d_1 # Subtract d1 as vertical offset from robot base
    
    x_c = sqrt( wc_dist_x**2 + wc_dist_z**2 )
    y_c = wc_dist_z

    L_25 = sqrt(x_c**2 + y_c**2)
    L_35 = sqrt(a_3**2+d_4**2)

    beta_4 = atan2(a_3,d_4)
    #beta_3 = acos((L_25**2-a_2**2-L_35**2)/(-2*L_35*a_2))
    cos_beta_3 = (( -L_25**2 + a_2**2 + L_35**2) / ( 2 * a_2 * L_35 ))
    beta_3 = atan2( cos_beta_3, sqrt( 1 - cos_beta_3**2 ))
    theta3 = np.pi/2 - beta_3 - beta_4
    
    beta_2 = atan2( y_c, x_c )
    #beta_1 = acos( ( L_35**2 - L_25**2 - a_2**2 ) / ( -2 * L_25 * a_2 ) )
    cos_beta_1 = ( ( -L_35**2 + L_25**2 + a_2**2 ) / ( 2 * L_25 * a_2 ) )
    beta_1 = atan2(  cos_beta_1, sqrt( 1 - cos_beta_1**2 ) ) 
    theta2 = np.pi/2 - beta_1 - beta_2

    print("theta1,2,3:",theta1, theta2, theta3)
    print(atan2(2.50022834193350,-0.512973099983402).evalf())
    
    return [ theta1.evalf(), theta2, theta3 ]

get_t123(1.8499, 0, 1.9466)

theta1,2,3: 0 0.114689449584584 1.95998482362893
1.77315854226795


[0, 0.114689449584584, 1.95998482362893]

once the first three joint variables are known, calculate 03R via application of homogeneous transforms up to the WC.

In [53]:
def get_t456(theta1, theta2, theta3, Rrpy):
 
    R0_3_eval = Matrix([[    sin(theta2 + theta3)*cos(theta1),   cos(theta1)*cos(theta2 + theta3),   -sin(theta1)],
                       [   sin(theta1)*sin(theta2 + theta3),   sin(theta1)*cos(theta2 + theta3),   cos(theta1)],
                       [   cos(theta2 + theta3),           -sin(theta2 + theta3),          0]])

    print("R0_3_eval:", R0_3_eval)

    R3_6_eval = R0_3_eval.inv() * Rrpy

    print("R3_6_eval:", R3_6_eval)

    theta5 = acos(R3_6_eval[1,2])
    theta6 = asin(R3_6_eval[1,1] / (-sin(theta5)))
    theta4 = asin(R3_6_eval[2,2] / sin(theta5))

    print("theta4,5,6:",theta4.evalf(),theta5.evalf(),theta6.evalf())
    
    return [ theta4, theta5, theta6 ]    

get_t456( 0, 0.114689449584584, 1.95998482362893, Rrpy )

R0_3_eval: Matrix([[0.875716781259277, -0.482825143318874, 0], [0, 0, 1], [-0.482825143318874, -0.875716781259277, 0]])
R3_6_eval: Matrix([[0.875716781259277, 0, -0.482825143318874], [-0.482825143318874, 0, -0.875716781259278], [0, 1, 0]])
theta4,5,6: 0 2.63771470717118 0


[0, 2.63771470717118, 0]

find a set of Euler angles corresponding to the rotation matrix

coose the correct solution among the set of possible solutions

In [7]:
# Calculate joint angles using Geometric IK method